In [4]:
import pathlib
from abc import ABC, abstractmethod
from typing import Callable, Optional, List
import pandas as pd
from pandas.errors import ParserError

class Handler(ABC):
    @abstractmethod
    def handle(self, *args, **kwargs):
        raise NotImplementedError("You should implement this!")
        
    def update_handler(self, new_handler: Callable) -> None:
        self.handle = new_handler
        
class FileHandler(Handler):
    _config = {'delimiter': ',',
              'header': 0,
              'on_bad_lines': 'error'}
  
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        try:
            return pd.read_csv(path ,delimiter=self._config['delimiter'],header=self._config['header'],on_bad_lines=self._config['on_bad_lines'])
        except ParserError as e:
            raise ParserError(str(e) + " perhaps setting delimiter=, header=, on_bad_lines= is in order.") from e

            return pd.DataFrame()
    
class Hitachi(Handler):
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        return pd.read_csv(path ,delimiter='\t',header=27,on_bad_lines='skip')

class Avantes(Handler):
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        return pd.read_csv(path,delimiter=';',decimal=',',header=5,on_bad_lines='skip')

class DataHandler(Handler):
    def __init__(self, handler: Optional[Callable]=None):
        super().__init__()
        
        if handler is not None:
            self.update_handler(handler)
        
    def handle(self,  data: pd.DataFrame) -> pd.DataFrame:
        return data

class Composite(DataHandler):
    def __init__(self) -> None:
        self.operations: List[DataHandler] = []
        
    def handle(self,  data: pd.DataFrame) -> pd.DataFrame:
        for o in self.operations:
            data = o.handle( data )
        return data

    def add(self, *operations: DataHandler) -> None:
        for o in operations:
            self.operations.append(o)

    def remove(self,*operations: DataHandler) -> None:
        for o in operations:
            self.operations.remove(o)
            
class DataModel(ABC):
    def __init__(self, path: pathlib.Path, reader: Optional[FileHandler]=FileHandler(), pipeline: Optional[DataHandler]=Composite()) -> None:
        self._reader=reader
        self._pipeline=pipeline
        self.path=path
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path: pathlib.Path):
        self._path = path
    @property
    def raw_data(self):
        try:
            return self._reader.handle(self.path)
        except ParserError as e:
            display(repr(e))
            return pd.DataFrame()
    @property
    def data(self):
        return self._pipeline.handle(self.raw_data)
    
        
dm = DataModel(path=Path('../patka-pomiary/CPE45_PFOBPy_comocat.txt'), reader=Hitachi())
display(dm.data[:5])
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
display(dm.data[:5])

# dm2 = DataModel(path=Path('./patka-pomiary/func39.txt'), reader=Avantes())
# display(dm2.data[:5])



,nm,Abs
0,1100.0,0.0420
1,1099.0,0.0420
2,1098.0,0.0419
3,1097.0,0.0418
4,1096.0,0.0417


,nm,Abs
0,1115.0,15.0420
1,1114.0,15.0420
2,1113.0,15.0419
3,1112.0,15.0418
4,1111.0,15.0417


In [ ]:
from abc import ABC, abstractmethod
from IPython.display import display, clear_output
from typing import List, Any, Optional
import numpy as np 
import pandas as pd 
from pathlib import Path
from lib.logger import log

class DisplayAPI(ABC):
    @abstractmethod
    def display(self):
        pass
    
class Print(DisplayAPI):
    def display(self, some_shit):
        print(some_shit.data)
    
class Display(DisplayAPI):
    def display(self, some_shit):
        display(some_shit.data)
    
class RawData(ABC):
    def __init__(self, file: Path, api: Optional[DisplayAPI] = Display() ) -> None:
        self.file = file
        self.api = api
        
        ###
        self.get_data(file)
     
    def display(self):
        self.api.display(self)
        
    @abstractmethod
    def get_data(self, file):
        pass
    
    @property
    def text(self):
        return self.file.read_text()
    @property
    def lines(self):
        return self.text.splitlines()

    
class CSVFile(RawData):
    def get_data(self,  file, header, delimiter):
        header_line, skip = header
        header = self.lines[header_line].split(delimiter)
        data = np.genfromtxt((x.replace(',', '.').encode() for x in open(file)),skip_header=header_line+skip, delimiter=delimiter)
        
        df = pd.DataFrame(data[:,1:], index=data[:,0], columns=header[1:])
        df.index.name=header[0]

        self.data=df
    
    def get_sections(self) -> List[Any]:
        indexes = [ *[k for k,line in enumerate(self.lines) if line==''], -1]
        return [self.lines[i+1:k] for i,k in zip(indexes[0::2], indexes[1::2])]

class Avantes(CSVFile):
    def get_data(self, file, header=(5,3), delimiter=';'):
        return super().get_data(file, header, delimiter)
        
class Hitachi(CSVFile):
    def get_data(self, file, header=(30,1), delimiter='\t'):
        return super().get_data(file, header, delimiter)
       
absorbance_csv =  Path('../patka-pomiary/CPE45_PFOBPy_comocat.txt')
fluorescence_csv =  Path('../patka-pomiary/func39.txt')

rd = Avantes( file=fluorescence_csv, api=Display() ) # api=Display() is set by default 
rd2 = Hitachi( file=absorbance_csv, api=Display() )

rd.display()
rd2.display()